# Building colocation table between metars and HRRRs

This will be used for training and evaluating the neural network.
Synchronized date and time. 
Metar wind gust as truth. 
All HRRR variables to be used in neural network. 
HRRR gust for comparison. 

In [1]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn import metrics

In [2]:
met_df = pd.read_pickle('metar_20190801_df.pkl')
met_df

,MetWindGust,Longitude,Latitude,Temperature,WindSpeed,WindDirection,Pressure
0,0.0,-110.85,31.42,297.05,0.0,0.0,101350.0
1,0.0,-110.96,32.13,300.35,2.6,170.0,101290.0
2,0.0,-111.17,32.14,299.15,2.1,70.0,0.0
3,0.0,-111.21,32.40,300.65,2.1,300.0,0.0
4,0.0,-116.98,32.57,290.95,1.5,0.0,101430.0
...,...,...,...,...,...,...,...
4389,0.0,-72.27,42.57,294.25,0.0,0.0,0.0
4390,0.0,-71.18,44.58,289.85,0.0,0.0,0.0
4391,0.0,-71.70,48.58,289.45,3.1,350.0,101660.0
4392,0.0,-73.34,49.28,284.85,0.0,0.0,101730.0


In [3]:
hrrr_df = pd.read_pickle('hrrr_20190801_df.pkl')
hrrr_df

,WindGust,Longitude,Latitude,Temperature,WindU,WindV,WindSpeed,PrecipRate
0,6.42368,-122.719528,21.138123,297.571167,-5.548723,-3.178455,6.248435,0.0
1,6.36118,-122.692861,21.145110,297.571167,-5.486223,-3.240955,6.248435,0.0
2,6.36118,-122.666190,21.152090,297.571167,-5.361223,-3.303455,6.185935,0.0
3,6.29868,-122.639515,21.159063,297.571167,-5.298723,-3.303455,6.123435,0.0
4,6.23618,-122.612835,21.166029,297.508667,-5.236223,-3.365955,6.060935,0.0
...,...,...,...,...,...,...,...,...
1905136,4.11118,-61.063480,47.883306,289.758667,-0.673723,1.446545,4.123435,0.0
1905137,4.23618,-61.026890,47.873043,289.758667,-0.798723,1.571545,4.248435,0.0
1905138,4.42368,-60.990312,47.862771,289.696167,-0.923723,1.634045,4.373435,0.0
1905139,4.48618,-60.953746,47.852488,289.633667,-1.048723,1.821545,4.498435,0.0


In [4]:
met_arr = met_df.to_numpy()
met_arr.shape

(4394, 7)

In [5]:
hrrr_arr = hrrr_df.to_numpy()
hrrr_arr.shape

(1905141, 8)

In [6]:
def closest_node(metar, hrrrs):
    closest_index = distance.cdist([metar], hrrrs).argmin()
    return hrrrs[closest_index]

hrrr_data = hrrr_arr
lookup_dict = {}

for i in range(0,2000): 
    metar_pt = (round(met_arr[i, 1],3), round(met_arr[i, 2],3))
    metar_gust = met_arr[i, 0]
    hrrr_pt = closest_node(metar_pt, hrrr_data[:, 1:3])
    hrrr_gust = hrrr_data[i, 0]
    hrrr_temp = hrrr_data[i, 3]
    hrrr_windu = hrrr_data[i, 4]
    hrrr_windv = hrrr_data[i, 5]
    hrrr_wind_speed = hrrr_data[i, 6]
    hrrr_precip_rate = hrrr_data[i, 7]
    lookup_dict[i] = {
        "metar_gust": metar_gust,
        "hrrr_gust": hrrr_gust,
        "metar_coords": metar_pt,
        "hrrr_coords_lon": round(hrrr_pt[0],3),
        "hrrr_coords_lat": round(hrrr_pt[1],3),
        "hrrr_temp": hrrr_temp,
        "hrrr_windu": hrrr_windu,
        "hrrr_windv": hrrr_windv,
        "hrrr_wind_speed": hrrr_wind_speed,
        "hrrr_precip_rate": hrrr_precip_rate,
    }

In [7]:
lookup_df = pd.DataFrame.from_dict(lookup_dict, orient ='index')
lookup_df

,metar_gust,hrrr_gust,metar_coords,hrrr_coords_lon,hrrr_coords_lat,hrrr_temp,hrrr_windu,hrrr_windv,hrrr_wind_speed,hrrr_precip_rate
0,0.0,6.42368,"(-110.85, 31.42)",-110.858,31.424,297.571167,-5.548723,-3.178455,6.248435,0.0
1,0.0,6.36118,"(-110.96, 32.13)",-110.951,32.144,297.571167,-5.486223,-3.240955,6.248435,0.0
2,0.0,6.36118,"(-111.17, 32.14)",-111.175,32.143,297.571167,-5.361223,-3.303455,6.185935,0.0
3,0.0,6.29868,"(-111.21, 32.4)",-111.222,32.408,297.571167,-5.298723,-3.303455,6.123435,0.0
4,0.0,6.23618,"(-116.98, 32.57)",-116.979,32.559,297.508667,-5.236223,-3.365955,6.060935,0.0
...,...,...,...,...,...,...,...,...,...,...
1995,0.0,1.86118,"(-89.53, 44.55)",-89.517,44.551,297.696167,0.013777,-1.865955,2.373435,0.0
1996,0.0,1.79868,"(-92.48, 44.58)",-92.495,44.593,297.696167,0.013777,-1.803455,2.310935,0.0
1997,0.0,1.79868,"(-88.73, 44.61)",-88.715,44.606,297.696167,0.013777,-1.803455,2.310935,0.0
1998,0.0,1.79868,"(-90.18, 44.63)",-90.189,44.618,297.696167,0.013777,-1.803455,2.310935,0.0


In [8]:
lookup_df.to_pickle('lookup_20190801_df.pkl')